In [26]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
df_2000 = pd.read_csv('../data_csv/clean_2000_df.csv')
df_2000.head()

,Unnamed: 0,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,0,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,1,180.56,182.30
1,1,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,2,180.82,182.10
2,2,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,3,180.61,181.21
3,3,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,4,180.24,179.31
4,4,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,5,181.06,177.70


In [27]:
# Drop the non-beneficial ID columns
df_2000 = df_2000.drop(columns = ['Unnamed: 0'], axis= 1)
df_2000.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,1,180.56,182.30
1,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,2,180.82,182.10
2,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,3,180.61,181.21
3,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,4,180.24,179.31
4,traditional,monthly,USA or Census Division,East North Central Division,DV_ENC,2008,purchase-only,5,181.06,177.70


In [28]:
#check for bucketing
# Generate our categorical variable list
cat_2000 = df_2000.dtypes[df_2000.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
df_2000[cat_2000].nunique()

hpi_type        3
hpi_flavor      2
frequency       4
level         162
place_name    162
yr              2
dtype: int64

In [29]:
#11 uniques values needs to be checked 
# Check the unique value counts to see if binning is required
df_2000.level.value_counts()

East North Central Division    20
Pacific Division               20
United States                  20
West North Central Division    20
South Atlantic Division        20
                               ..
Fresno, CA                      4
Gary, IN (MSAD)                 4
Grand Rapids-Kentwood, MI       4
Greensboro-High Point, NC       4
Puerto Rico                     4
Name: level, Length: 162, dtype: int64

In [30]:
#there are a substantial number of datapoints. leave it alone no need to bucket
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_2000[cat_2000]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(cat_2000)
encode_df.head()

C:\Users\renee\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,hpi_type_developmental,hpi_type_distress-free,hpi_type_traditional,hpi_flavor_monthly,hpi_flavor_quarterly,frequency_MSA,frequency_Puerto Rico,frequency_State,frequency_USA or Census Division,"level_Akron, OH",...,place_name_USA,place_name_UT,place_name_VA,place_name_VT,place_name_WA,place_name_WI,place_name_WV,place_name_WY,yr_expanded-data,yr_purchase-only
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [31]:
# Merge one-hot encoded features and drop the originals
df_2000 = df_2000.merge(encode_df,left_index=True, right_index=True)
df_2000 = df_2000.drop(cat_2000,1)
df_2000.head()

C:\Users\renee\AppData\Local\Temp\ipykernel_62352\3171705586.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_2008 = df_2008.drop(cat_2008,1)


,place_id,period,index_nsa,index_sa,hpi_type_developmental,hpi_type_distress-free,hpi_type_traditional,hpi_flavor_monthly,hpi_flavor_quarterly,frequency_MSA,...,place_name_USA,place_name_UT,place_name_VA,place_name_VT,place_name_WA,place_name_WI,place_name_WV,place_name_WY,yr_expanded-data,yr_purchase-only
0,2008,1,180.56,182.30,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2008,2,180.82,182.10,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2008,3,180.61,181.21,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2008,4,180.24,179.31,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2008,5,181.06,177.70,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
# Remove loan status target from features data
y = df_2000.hpi_type_traditional
X = df_2000.drop(columns=["index_sa","index_nsa"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 1.000


In [34]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
30/30 [==============================] - 1s 2ms/step - loss: 0.8600 - accuracy: 0.4148
Epoch 2/50
30/30 [==============================] - 0s 3ms/step - loss: 0.3791 - accuracy: 0.9048
Epoch 3/50
30/30 [==============================] - 0s 3ms/step - loss: 0.1864 - accuracy: 0.9767
Epoch 4/50
30/30 [==============================] - 0s 2ms/step - loss: 0.1016 - accuracy: 0.9884
Epoch 5/50
30/30 [==============================] - 0s 3ms/step - loss: 0.0644 - accuracy: 0.9894
Epoch 6/50
30/30 [==============================] - 0s 3ms/step - loss: 0.0444 - accuracy: 0.9926
Epoch 7/50
30/30 [==============================] - 0s 3ms/step - loss: 0.0320 - accuracy: 0.9968
Epoch 8/50
30/30 [==============================] - 0s 3ms/step - loss: 0.0232 - accuracy: 0.9989
Epoch 9/50
30/30 [==============================] - 0s 3ms/step - loss: 0.0173 - accuracy: 1.0000
Epoch 10/50
30/30 [==============================] - 0s 3ms/step - loss: 0.0133 - accuracy: 1.0000
Epoch 11/50
30/30 [